In [1]:
import pandas as pd
import ivolatility as ivol
import os

# Script description: https://www.ivolatility.com/api/docs#section/Python-Examples/Example-No.-3-Back-Testing

ivolApiKey = os.getenv('API_KEY')
ivol.setLoginParams(apiKey=ivolApiKey)

etf = 'SPY'
moneyness = 0
delta = 0.5
dte = 90
cp = 'C'
from_ = '2021-01-01'
to = '2025-10-30'
inPosition = False
entryPrice = 0
realizedPnl = 0
totalRealizedPnl = 0
unrealizedPnl = 0
lowerBound = 0.15
upperBound = 0.20

getOptionIds = ivol.setMethod('/equities/eod/nearest-option-tickers')
getMarketData = ivol.setMethod('/equities/eod/single-stock-option-raw-iv')

print(etf)

optionIds = getOptionIds(symbol=etf, dte=dte, moneyness=moneyness, delta=delta, callPut=cp)
print(optionIds)

prev_optionId = None
for optionId in optionIds['option_id']:
  if optionId == prev_optionId:
    continue
  prev_optionId = optionId
  #load options data
  marketData = getMarketData(optionId=optionId, from_=from_, to=to)
  optionSymbol = optionIds.iloc[optionIds.index[optionIds['option_id'] == optionId][0]]['option_symbol']

  #backtest
  print(optionId, optionSymbol)
  for i in range(marketData.shape[0]):
    #if not in a position, check iv to get into a position
    if not inPosition:
      if marketData.iloc[i]['iv'] < lowerBound:
        entryPrice = marketData.iloc[i]['ask']
        inPosition = True
      elif marketData.iloc[i]['iv'] > upperBound:
        entryPrice = -marketData.iloc[i]['bid']
        inPosition = True
    #if in a position, update unrealized pnl and check if conditions warrant exiting trade
    elif inPosition:
      if entryPrice > 0:
        if marketData.iloc[i]['iv'] > upperBound:
          realizedPnl = marketData.iloc[i]['bid'] - entryPrice
          totalRealizedPnl += realizedPnl
          inPosition = False
          entryPrice = 0
          unrealizedPnl = 0
        else:
          unrealizedPnl = marketData.iloc[i]['bid'] - entryPrice
      elif entryPrice < 0:
        if marketData.iloc[i]['iv'] < lowerBound:
          realizedPnl = marketData.iloc[i]['ask'] - entryPrice
          totalRealizedPnl += realizedPnl
          inPosition = False
          entryPrice = 0
          unrealizedPnl = 0
        else:
          unrealizedPnl = marketData.iloc[i]['ask'] - entryPrice

      if not inPosition:
        print(f'PNL ::: date: {marketData.iloc[i]["date"]}, IV = {marketData.iloc[i]["iv"]}, PNL = {round(realizedPnl,2)}')
  result = 'Total realized PNL is ' + str(round(totalRealizedPnl,2))
  if inPosition:
    result += ' and an open position with unrealized PNL of ' + str(round(unrealizedPnl,2))
  print(result)

SPY
   option_id          option_symbol comment
0  133168177  SPY   250630C00559000        
1  133168179  SPY   250630C00560000        
133168177 SPY   250630C00559000
PNL ::: date: 2024-12-18, IV = 0.201991, PNL = 16.76
Total realized PNL is 16.76 and an open position with unrealized PNL of 76.39
133168179 SPY   250630C00560000
PNL ::: date: 2024-07-26, IV = 0.1478, PNL = 90.75
Total realized PNL is 107.51 and an open position with unrealized PNL of 75.78
